# Amazon Review Data Pipeline

The goal of this jupyter notebook is to:
1. Load Amazon Review data acquired from [https://nijianmo.github.io/amazon/index.html](https://nijianmo.github.io/amazon/index.html)
2. Turn it into a Spark RDD
3. Create a Bag-of-Words with:
    - Single words
    - 1-grams and 2-grams
4. Visualize the frequency of these words occurences in the Review data
5. Create a LabeledPoint RDD for each review and save it as a libsvm file.

## Preparing Data

The data cannot be read directly from it's gzipped format into a Spark DataFrame. It produces Schema errors because there are a few naming collisions in the Json data.

Instead, the data must be unzipped and a few of its errors must be correct before being loaded.

The following script will achieve that. Specifically, it will ensure consistent casing for the field names 'style' and 'styleName'.

Note: it is possible that both `gzip` and `sed` will need to be installed.

In [51]:
%%bash

# Decompress Gzip
gzip -dk $PWD/data/Musical_Instruments_5.json.gz

# # Replace problematic field names
sed -i "s:style Name:styleName:" $PWD/data/Musical_Instruments_5.json
sed -i "s:style name:styleName:" $PWD/data/Musical_Instruments_5.json
sed -i "s:Style:style:" $PWD/data/Musical_Instruments_5.json

gzip: /home/steph/proj/550/project/data/Musical_Instruments_5.json already exists;	not overwritten


## Imports

In [55]:
from pyspark.sql import SparkSession
import gzip
import json
import re
# import matplotlib.pyplot as plt
# import seaborn as sns

## Spark DataFrame

The goal of this section is to convert the Pandas dataframe from above into a Spark dataframe, to drop all columns except 'overall' (rating) and 'reviewText' and create an RDD from this data.

#### Creating Spark Session

In [13]:
spark = SparkSession.builder \
    .master("local[16]") \
    .appName("data_pipeline") \
    .getOrCreate()

22/12/13 14:19:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


#### Creating Spark Dataframe

In [53]:
spark_df = spark.read.json('data/Musical_Instruments_5.json')
spark_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color Name:: string (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Configuration:: string (nullable = true)
 |    |-- Edition:: string (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Item Display Length:: string (nullable = true)
 |    |-- Item Package Quantity:: string (nullable = true)
 |    |-- Length:: string (nullable = true)
 |    |-- Model Number:: string (nullable = true)
 |    |-- Number of Items:: string (nullable = true)
 |    |-- Package Quantity:: string (nullable = true)
 |    |-- Package Type:: string (nullable = true)
 |    |-- Platform for Display:

In [54]:
print(f"There are {spark_df.count()} rows of data")

There are 231392 rows of data


## Cleaning Data

1. Drop all columns except `overall` and `reviewTest`
2. Drop all rows with NA values
3. Convert to RDD
4. Convert to lowercase
5. Strip non-alphabetic/space chars
6. Split into vector of strings
7. Remove empty reviews
8. Simplify rating into 0 or 1

In [57]:
# Drop all columns except reviewText and overall
df = spark_df[['overall', 'reviewText']]
df.printSchema()

# Drop all rows with NA values
df = df.dropna()

# Convert to rdd
rdd = df.rdd

# Convert to lowercase
rdd = rdd.map(lambda x: (x[0], x[1].lower()))

# Strip non-alphanumeric & space chars
rdd = rdd.map(lambda x: (x[0], re.sub(r'[^A-Za-z ]+', '', x[1])))

# Convert review to array of words
rdd = rdd.map(lambda x: (x[0], x[1].split()))

# Convert overall rating to 1 or 0
# 4-5 -> Positive
# 1-3 -> Negative
rdd = rdd.map(lambda x: (1, x[1]) if x[0] in [4,5] else (0, x[1]))

# Filter out empty reviews
basic_rdd = rdd.filter(lambda x: len(x[1]) > 0)

# Cache as basic_rdd
basic_rdd.cache
basic_rdd.count()

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)



231273

## Create Bag-of-Words of 1-Grams

### Method for Filtering Stop Words

Stopwords found [here](https://kavita-ganesan.com/what-are-stop-words/) and taken from [here](http://snowball.tartarus.org/algorithms/english/stop.txt).

In [113]:
stopwords = []

with open("data/stopwords.txt") as file:
    global stopwords
    stopwords = file.read().splitlines()

def remove_stopwords(x):
    global stopwords
    result = []
    for s in x[1]:
        if not s in stopwords:
            result.append(s)
    return (x[0], result)
    
# def filter_stopwords(x):
#     global stopwords
#     result = []
#     for s in x[1]:
#         if isinstance(s, str):
#             if not s in stopwords:
#                 result.append(s)
#             return (x[0], result)
#         if not s[0] in stopwords or not s[-1] in stopwords:
#             result.append(s)
#     return (x[0], result)

In [115]:
onegram_flat_rdd = basic_rdd.map(remove_stopwords).flatMap(lambda x: x[1])
onegram_rdd = onegram_flat_rdd.distinct()

In [116]:
word_counts = onegram_flat_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).collect()

In [117]:
print(f"There are {onegram_rdd.count()} unique words in the corpus")

There are 156073 unique words in the corpus


In [133]:
onegram_counts_rdd = onegram_flat_rdd \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda a,b: a+b)

In [134]:
onegram_counts_rdd.cache

<bound method RDD.cache of PythonRDD[218] at RDD at PythonRDD.scala:53>

In [131]:
print(f"There are {onegram_shared_rdd.count()} shared words in the corpus")

There are 93 shared words in the corpus


In [139]:
onegram_counts_rdd.filter(lambda x: x[1] > 10000).map(lambda x: (-x[1], x[0])).sortByKey().take(20)

[(-79048, 'great'),
 (-63701, 'good'),
 (-59495, 'guitar'),
 (-55327, 'sound'),
 (-54859, 'one'),
 (-53253, 'like'),
 (-49326, 'just'),
 (-48112, 'use'),
 (-43823, 'can'),
 (-43285, 'strings'),
 (-37369, 'will'),
 (-36488, 'get'),
 (-33601, 'price'),
 (-31183, 'really'),
 (-29998, 'quality'),
 (-29495, 'works'),
 (-29208, 'nice'),
 (-27165, 'little'),
 (-26956, 'dont'),
 (-25953, 'much')]

## Convert to N-Grams

In [74]:
basic_rdd.count()

231307

In [71]:
def ngram(n: int, x: [str]):
    if n == 1:
        return x
    result = []
    
    for i in range(len(x)-n+1):
        item = []
        a = i
        for a in range(i, i+n):
            item.append(x[a])
        result.append(item)
    return result

In [72]:
ngram_rdd = basic_rdd.filter(lambda x: x[0] == 0)

print(f"Num neg: {ngram_rdd.count()}")

n = 4
ngram_rdd = ngram_rdd.map(lambda x: (x[0], ngram(n, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)

# ngram_rdd_nosw = ngram_rdd.map(lambda x: (x[0], filter_stopwords(x[1])))
# ngram_counts_rdd = ngram_rdd.map(lambda x: len(x[1]))
# ngram_nosw_count_rdd = ngram_rdd_nosw.map(lambda x: len(x[1]))

# def sum(a, b):
#     return a+b

if n == 1:
    ngrams_rdd = ngram_rdd.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
else:
    ngrams_rdd = ngram_rdd.flatMap(lambda x: x[1]).map(lambda x: (tuple(x), 1))
ngram_counts = ngrams_rdd.reduceByKey(sum)

print(type(ngram_counts))

shared_grams = ngram_counts.filter(lambda x: x[1] > 100)

# # count_total = ngrams_rdd.count()
print(f"total ngrams:  {ngram_counts.count()}")
print(f"shared ngrams: {shared_grams.count()}")

for a in shared_grams.sortByKey().collect():
    print(a)

# ngrams_counts = ngrams_rdd.countByKey()

# distinct_ngrams = ngrams_rdd.toDF().distinct()
# print(f"distinct ngrams: {distinct_ngrams.count()}")

# print(f"No SW Filter:\n\tmin:  {ngram_counts_rdd.min()}\n\tmax:  {ngram_counts_rdd.max()}\n\tmean: {ngram_counts_rdd.mean()}")
# print(f"With SW Filter:\n\tmin:  {ngram_nosw_count_rdd.min()}\n\tmax:  {ngram_nosw_count_rdd.max()}\n\tmean: {ngram_nosw_count_rdd.mean()}")

Num neg: 30768
<class 'pyspark.rdd.PipelinedRDD'>
22/12/13 15:09:26 ERROR Executor: Exception in task 1.0 in stage 24.0 (TID 229)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 540, in func
    return f(iterator)
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 24.0 failed 1 times, most recent failure: Lost task 1.0 in stage 24.0 (TID 229) (172.17.16.130 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/steph/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "

In [176]:
one_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(1, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
two_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(2, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
three_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(3, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)
four_gram_rdd = basic_rdd.map(lambda x: (x[0], ngram(4, x[1]))).map(filter_stopwords).filter(lambda x: len(x[1]) > 0)

# print("Reviews w/ 1-grams: " + str(one_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 2-grams: " + str(two_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 3-grams: " + str(three_gram_rdd.filter(lambda x: x[0] == 0).count()))
# print("Reviews w/ 4-grams: " + str(four_gram_rdd.filter(lambda x: x[0] == 0).count()))

print("Reviews w/ 1-grams: " + str(one_gram_rdd.count()))
print("Reviews w/ 2-grams: " + str(two_gram_rdd.count()))
print("Reviews w/ 3-grams: " + str(three_gram_rdd.count()))
print("Reviews w/ 4-grams: " + str(four_gram_rdd.count()))

# for a in rdd.take(3):
    # print(a)

Reviews w/ 1-grams: 230993


Reviews w/ 2-grams: 222503


Reviews w/ 3-grams: 209881


Reviews w/ 4-grams: 203566


In [94]:
filter_stopwords(["a"])

157


In [66]:
rdd = convert_to_n_grams(2, basic_rdd)

4
14
12
133
4
6
62
25
19
27
4
50
4
14
12
7
3
8838

56
10
14
6
89
33
2
29
28
7
9
8
3173

30
37
70
23
118
8
240
105
21
23
5
20
4
4
18
46
4
8
282
32
34
90
42
98
22
56
71
250
6
7
33
14
312
83

28
70
4
22
16
2
237
29
21
10
96
20
23
94
51
84
61
81
84
2
75
38
250
114
10
7622

2
58
2
137
25
7
30
90
5
18
88
174

29
51
4
3
78
5
9
8
6
41
1
29
3
136
2
7
46
68
25
3
105
21
46
43
182
119
49
42
40
45
67
23
146
32
38
59
14
10
16


None
None
None


In [46]:
df.write.json("data/Musical_Instruments_2.json")

In [48]:
df = spark.read.json("data/Musical_Instruments_2.json")

In [51]:
df.printSchema()
print(df.count())

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)

231344
231344
